In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl

def prepare_dataframe(infile, filters={'test_pos': 2}, renaming=None):
    '''
    Reads a dataframe, filters it and puts human-friendly names in some default entries
    plus additional ones specified by the renaming parameter: dict(original->new_name)
    '''
    df = pd.read_csv(infile)
    df = df.rename(columns = {'strat':'portfolio'}) # renames the portfolio column

    # display(df.describe())
    
    # human-friendly portfolio names
    df.replace({'WR_LR_RR_HR': 'rush', 'WR_LR_RR_HR_BB_BK': 'rush+suport', 'WR_LR_RR_HR_WD_LD_RD_HD': 'rush+defense',  'WR_LR_RR_HR_WD_LD_RD_HD_BB_BK': 'all'}, inplace=True)
    
    # human-friendly feature selectors
    df.replace({'materialdistancehp': 'Simple', 'quadrantmodel': 'Per-quadrant'}, inplace=True)
    
    #if there is any other rename, do it:
    if renaming is not None:
        df.replace(renaming, inplace=True)
        
    return filter_data(df,filters)

def filter_data(data, filters):
    '''
    Excludes data from the dataframe according to user criteria,
    passed as a dict(colname -> value)
    '''
    for fname,fvalue in filters.items():
        if isinstance(fvalue, float):
            #print(fname, fvalue, df[fname])
            data = data[np.isclose(data[fname],fvalue)] 
        else:
            data = data[data[fname] == fvalue] 
        # drops the column
        data.drop(fname, axis=1, inplace=True)
    return data


def map_feature_portfolio(infile, metric='%score', 
                          mean_across_maps=False, filters={'test_pos': 2}, 
                          unstack_level = 2, renaming=None):
    
    df = prepare_dataframe(infile, filters, renaming)
    
    # the columns I want to see
    #indexes = ['mapname', 'feature', 'portfolio', 'alpha', '_lambda', 'train_matches', 'dec_int', 'test_pos']
    indexes = ['train_matches', 'mapname', 'feature', 'portfolio', 'alpha', '_lambda', 'dec_int', 'test_pos']
    
    # removes filter keys from the indexes, as they're no longer part of the data frame
    for fname,fvalue in filters.items():
        indexes.remove(fname)
        
    # display(df.describe())
    
    # builds a hierarchical (multiindex) data frame (kinda groups by map, strategy & units)
    hier = df.set_index(indexes) 
    #hier.index = natsorted(hier.index)
    #print(np.where(hier.index.duplicated()))  # finds entries with duplicate indexes, thx: https://stackoverflow.com/a/28652153/1251716
    
    # unstack the hierarchical dataframe up to the portfolio (all other parameters are grouped?)
    unstacked = hier.unstack(unstack_level)[metric]
    
    if mean_across_maps: 
        # if i group by the 'feature' level, i can get the mean on all maps played using that feature extractor
        return unstacked.groupby(level=['feature']).mean()
    else:
        return unstacked
        
        
def autolabel(ax):
    """
    Attach a text label above each bar displaying its height 
    (source: https://stackoverflow.com/a/22689127/1251716)
    """
    rects = [rect for rect in ax.get_children() if isinstance(rect, mpl.patches.Rectangle)]
    for rect in rects:
        height = rect.get_height()
        if height != 1: #little hack to avoid a dummy 1 between bars
            ax.text(rect.get_x() + rect.get_width()/2., 1.001*height,
                    '%d' % int(height),
                    ha='center', va='bottom')

In [ ]:
# displays the results for all adversaries
opponents = ['A3N', 'SSSmRTS','StrategyTactics','PuppetSearchMCTS','WorkerRush','LightRush'] 

# filters by a specific set of parameters
filters = {'test_pos': 2, 'dec_int': 100, 'train_matches': 100000, '_lambda': 0.5, 'alpha': 0.01,}

for o in opponents:
    print("---{}, no search budget---".format(o))
    display(map_feature_portfolio('../results/parameters/{}_avg.csv'.format(o), '%score', True, filters))
    
    print("---{}, search budget=100ms---".format(o))
    display(map_feature_portfolio('../results/parameters/{}_b100_avg.csv'.format(o), '%score', True, filters))

In [ ]:
### ONLY LIGHT & WORKER RUSH WITH b0 and b100:
# filters by a specific set of parameters
filters = {'test_pos': 2, 'dec_int': 100,  '_lambda': 0.5, 'alpha': 0.01, 'portfolio': 'all'}

for o in ['WorkerRush', 'LightRush']:
    for b in [0, 100]:
        print("---{}, search budget={}ms---".format(o, b))
        display(map_feature_portfolio('../results/parameters/{}_b{}_avg.csv'.format(o,b), '%score', False, filters))

In [ ]:
### This is to check the feature extractor on regular maps
import matplotlib.pyplot as plt

print('# Comparing feature extractors on "regular" maps')
# filters by a specific set of parameters
#filters = {'test_pos': 2, 'dec_int': 100,  '_lambda': 0.5, 'alpha': 0.01, 'portfolio': 'all',}# 'train_matches': 100000}

#for o in ['WorkerRush', 'LightRush']:
#    print("---{}---".format(o))
#    display(map_feature_portfolio('../results/parameters/{}_b0_avg.csv'.format(o), '%score', False, filters, 1))
    
#print('# Comparing feature extractors, now only at the end of training ')

filters = {'test_pos': 2, 'dec_int': 100,  '_lambda': 0.5, 'alpha': 0.01, 'portfolio': 'all', 'train_matches': 100000}
for o in ['WorkerRush', 'LightRush']:
    #print("---{}---".format(o))
    data = map_feature_portfolio('../results/parameters/{}_b0_avg.csv'.format(o), '%score', False, filters, 1)
    display(data)
    data.sort_values(by=['mapname'], ascending=False, inplace=True) # so that 8x8 comes before 32x32
    ax = data.plot(title=o, ylim=[0, 105],kind='bar')
    ax.set(xlabel="Map", ylabel="Score (%)")
    ax.set_xticklabels( ax.get_xticklabels(), rotation=0) # rotates the labels without changing them
    autolabel(ax)
    ax.get_figure().savefig(f'/tmp/feature-vs_{o}_regular.pdf')

In [ ]:
### This is to check the feature extractor on irregular/split maps

print('# Comparing feature extractors on "irregular/split" maps')

# filters by a specific set of parameters
#filters = {'test_pos': 2, 'dec_int': 100,  '_lambda': 0.5, 'alpha': 0.01, 'portfolio': 'all',}# 'train_matches': 100000}

#for o in ['WorkerRush', 'LightRush']:
#     print("---{}---".format(o))
#     display(map_feature_portfolio('../results/features/{}_b0_avg.csv'.format(o), '%score', False, filters, 1))
    
# print('# Comparing feature extractors, now only at the end of training ')

filters = {'test_pos': 2, 'dec_int': 100,  '_lambda': 0.5, 'alpha': 0.01, 'portfolio': 'all', 'train_matches': 100000}
map_rename = {'FourBasesWorkers8x8': '8x8', 'NoWhereToRun9x8': '9x8', 'TwoBasesBarracks16x16': '16x16'}

for o in ['WorkerRush', 'LightRush']:
    data = map_feature_portfolio('../results/features/{}_b0_avg.csv'.format(o), filters=filters, unstack_level= 1, renaming=map_rename)
    #print("---{}---".format(o))
    #display(data)
    data = data.loc[['8x8', '9x8', '16x16']] # to sort the columns for the barplot (https://stackoverflow.com/a/20555406/1251716)
    ax = data.plot(title=o, ylim=[0, 105],kind='bar')
    ax.set(xlabel="Map", ylabel="Score (%)")
    ax.set_xticklabels( ax.get_xticklabels(), rotation=0) # rotates the labels without changing them
    autolabel(ax)
    ax.get_figure().savefig(f'/tmp/feature-vs_{o}_irregular.pdf')

In [ ]:
'''
This cell is to plot line charts with %score according to search budget, 
train matches, map and opponent
'''

import matplotlib.pyplot as plt

# filters by a specific set of parameters
filters = {'test_pos': 2, 'dec_int': 100,  '_lambda': 0.5, 'alpha': 0.01, 'portfolio': 'all', 'feature': 'Simple'}

data_array = []
for o in ['WorkerRush', 'LightRush']:
    opp_data = []
    print(f'---{o}---')
    for b in [0, 100]:
        data = prepare_dataframe(f'../results/parameters/{o}_b{b}_avg.csv', filters=filters)
        
        # augments the dataset with the budget column
        data['budget'] = [b] * len(data)
        #display(data)
        opp_data.append(data)
    data_array += opp_data
    print(f'len(opp-data): {len(opp_data)}')
    
    df = pd.concat(opp_data) # creates a dataframe with the opponent data on both maps, it is processed below
    # here I have data for a test opponent with all desired search budgets, will try plotting per map
    #display(data)
    for mapname in ['basesWorkers8x8', 'basesWorkers32x32A']:
        print(f'\t---{mapname}----')
        # get data from a single map & remove the map column
        per_map = df[df['mapname'] == mapname] 
        per_map.drop('mapname', axis=1, inplace=True)
        #display(per_map)
        hier = per_map.set_index(['train_matches', 'budget']) 
        display(hier)
        print(np.where(hier.index.duplicated())) # prints where hierarchical has duplicate entries (cannot unstack)
        unstacked = hier.unstack(0)['%score'] # groups(?) results via score
        #display(unstacked)
        ax = unstacked.plot(title=f'{o} - {mapname}', ylim=[-1, 105], xlim=[-1, 101],style=['*-','o-','+-','s-'])
        ax.set(xticks=[0,100]) # if there's data on other budgets, put here
        ax.set(xlabel="Search budget (ms)", ylabel="Score (%)")
        ax.get_figure().savefig(f'/tmp/budget-{mapname}_vs_{o}.pdf')

'''
all_data = pd.concat(data_array) #now I have data from all opponents, search budgets & maps
#display(all_data) 
fig, axes = plt.subplots(nrows=1,ncols=2, figsize=(6,4), sharex=True)
# filters out by each map
for mapname in ['basesWorkers8x8', 'basesWorkers32x32A']:
    print(f'\t---{mapname}----')
    # get data from a single map & remove the map column
    per_map = all_data[all_data['mapname'] == mapname] 
    per_map.drop('mapname', axis=1, inplace=True)
    
    # loop over opponents
    for i,o in enumerate(['WorkerRush', 'LightRush']):
        per_map_opp = per_map[per_map["test_opp"] == o]
        axes[i].plot(per_map_opp["budget"], per_map_opp["%score"], label=o)
    #axes.margins(x=0)
        axes[i].set_title(o)
'''
    
#     display(per_map)

#     # builds a hierarchical (multiindex) data frame 
#     hier = per_map.set_index(['train_matches', 'budget', 'test_opp']) 
#     unstacked = hier.unstack(2)['%score'] # groups(?) results via score
#     wr_budget = unstacked.groupby('budget')
#     display(unstacked)
#     unstacked.plot()
pass

In [8]:
'''
This cell is to create tables with results vs state of the art algorithms with materialadvantage model
'''
mpl.use('TkAgg')
from collections import OrderedDict

filters = {
    'test_pos': 2, 'dec_int': 100,  '_lambda': 0.5, 'alpha': 0.01, 
    'portfolio': 'all', 'feature': 'Simple'
}

opponents = ['A3N', 'LightRush', 'PuppetSearchMCTS', 
             'SSSmRTS', 'StrategyTactics', 'WorkerRush']

# considers only b=0
data_array = [prepare_dataframe(f'../results/parameters/{o}_b0_avg.csv', filters=filters) for o in opponents]
df = pd.concat(data_array)

# plot-friendly opponent names
opp_mapping = OrderedDict({
    'LightRush': 'LR', 'WorkerRush': 'WR', # baselines
    'PuppetSearchMCTS': 'PS', 'StrategyTactics': 'ST', # puppet-based 
    'A3N': 'A3N', 'SSSmRTS': 'SSS', # SOTA (A3N is mapped to itself because this is used for sorting too)
})
df.replace(opp_mapping, inplace=True)

# sorts the opponents -- hint by: https://stackoverflow.com/a/27255567/1251716
df.test_opp = df.test_opp.astype("category")
df.test_opp.cat.set_categories(opp_mapping.values(), inplace=True)
# display(df)
#df = df.loc[['basesWorkers8x8', 'basesWorkers16x16A', 'basesWorkers32x32A']] ## attempt to sort map names

hier = df.set_index(['test_opp', 'mapname']) 
unstacked = hier.unstack(0)['%score']
ax = unstacked.plot(kind='bar',title='All maps')
#ax.get_figure().show()
#display(unstacked)

#now, try to separate 'regular' vs 'split' maps
all_maps = {
    'Regular': ['basesWorkers8x8', 'basesWorkers16x16A', 'basesWorkers24x24A', 'basesWorkers32x32A'],
    'Split': ['FourBasesWorkers8x8', 'NoWhereToRun9x8', 'TwoBasesBarracks16x16', 'DoubleGame24x24'] 
}

for map_type, map_names in all_maps.items():
    print(f'---- {map_type} ----')
    filtered = df[df['mapname'].isin(map_names)]  # isin hint: https://stackoverflow.com/a/12098586/1251716
    # sorts the filtered dataframe by the order given in map_names
    filtered.mapname = filtered.mapname.astype("category")
    filtered.mapname.cat.set_categories(map_names, inplace=True)
    #display(filtered)
    
    hier = filtered.set_index(['test_opp', 'mapname']) 
    unstacked = hier.unstack(0)['%score']
    ax = unstacked.plot(kind='bar', title=map_type)
    ax.set_xticklabels( ax.get_xticklabels(), rotation=0) # rotates the labels without changing them
    autolabel(ax)
    #ax.get_figure().show()
    ax.get_figure().savefig(f'/tmp/materialadv_{map_type}.pdf')
    display(unstacked)

    
# BWDistant resources is neither regular nor split: it is open as a regular map, but resources are distant 



# # regular maps:
# for mapname in ['basesWorkers8x8', 'basesWorkers16x16A', 'basesWorkers32x32A']:
#     # filters by map & drops the map column
#     per_map = df[df['mapname'] == mapname] 
#     per_map.drop('mapname', axis=1, inplace=True)
    
#     print(f'---- {mapname} ----')
#     display(per_map)
    
#     hier = per_map.set_index(['test_opp']) 
#     #hier.plot()
#     unstacked = hier.unstack(0)['%score']
#     display(unstacked)
    
#     ax = unstacked.plot(kind='bar', title=mapname)
#     ax.set(xlabel="Opponent", ylabel="Score (%)")
#     ax.set_xticklabels( ax.get_xticklabels(), rotation=0) # rotates the labels without changing them
#     autolabel(ax)
#     ax.get_figure().savefig(f'/tmp/wins_{mapname}.pdf')

#the code below is to verify which opponents were not tested within the given parameters
# for o in opponents:
#     print(f'--- {o} ---')
#     display(prepare_dataframe(
#         f'../results/parameters/{o}_b100_avg.csv', 
#         filters={'test_pos': 2, 'dec_int': 100, '_lambda': 0.5, 'alpha': 0.01, 'portfolio': 'all', 'feature': 'Simple'}
#     ))


/home/anderson/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


---- Regular ----


/home/anderson/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


test_opp,LR,WR,PS,ST,A3N,SSS
mapname,,,,,,
basesWorkers8x8,100.000000,33.333333,100.000000,55.833333,99.166667,66.666667
basesWorkers16x16A,33.333333,33.333333,45.000000,66.666667,31.666667,3.333333
basesWorkers24x24A,33.333333,66.666667,31.666667,95.000000,20.000000,0.000000
basesWorkers32x32A,66.666667,100.000000,63.333333,98.333333,64.166667,65.000000


---- Split ----


/home/anderson/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


test_opp,LR,WR,PS,ST,A3N,SSS
mapname,,,,,,
FourBasesWorkers8x8,33.333333,0.000000,81.666667,60.833333,45.000000,0.000000
NoWhereToRun9x8,50.000000,50.000000,0.000000,45.000000,12.500000,44.166667
TwoBasesBarracks16x16,0.000000,66.666667,66.666667,39.166667,48.333333,31.666667
DoubleGame24x24,50.000000,25.000000,58.750000,62.500000,33.750000,52.500000


In [10]:
'''
This cell is to create tables with results vs state of the art algorithms with **quadrantmodel**
'''
mpl.use('TkAgg')
from collections import OrderedDict

filters = {
    'test_pos': 2, 'dec_int': 100,  '_lambda': 0.5, 'alpha': 0.01, 
    'portfolio': 'all', 'feature': 'Per-quadrant'
}

opponents = ['A3N', 'LightRush', 'PuppetSearchMCTS', 
             'SSSmRTS', 'StrategyTactics', 'WorkerRush']

# considers only b=0
data_array = [prepare_dataframe(f'../results/summary_quadrantmodel/{o}_b0_avg.csv', filters=filters) for o in opponents]
df = pd.concat(data_array)

# plot-friendly opponent names
opp_mapping = OrderedDict({
    'LightRush': 'LR', 'WorkerRush': 'WR', # baselines
    'PuppetSearchMCTS': 'PS', 'StrategyTactics': 'ST', # puppet-based 
    'A3N': 'A3N', 'SSSmRTS': 'SSS', # SOTA (A3N is mapped to itself because this is used for sorting too)
})
df.replace(opp_mapping, inplace=True)

# sorts the opponents -- hint by: https://stackoverflow.com/a/27255567/1251716
df.test_opp = df.test_opp.astype("category")
df.test_opp.cat.set_categories(opp_mapping.values(), inplace=True)
#display(df)
#df = df.loc[['basesWorkers8x8', 'basesWorkers16x16A', 'basesWorkers32x32A']] ## attempt to sort map names

hier = df.set_index(['test_opp', 'mapname']) 
unstacked = hier.unstack(0)['%score']
ax = unstacked.plot(kind='bar',title='All maps')
#ax.get_figure().show()
#display(unstacked)

#now, try to separate 'regular' vs 'split' maps
all_maps = {
    'Regular': ['basesWorkers8x8', 'basesWorkers16x16A', 'basesWorkers24x24A', 'basesWorkers32x32A'],
    'Split': ['FourBasesWorkers8x8', 'NoWhereToRun9x8', 'TwoBasesBarracks16x16', 'DoubleGame24x24'] 
}

for map_type, map_names in all_maps.items():
    print(f'---- {map_type} ----')
    filtered = df[df['mapname'].isin(map_names)]  # isin hint: https://stackoverflow.com/a/12098586/1251716
    # sorts the filtered dataframe by the order given in map_names
    filtered.mapname = filtered.mapname.astype("category")
    filtered.mapname.cat.set_categories(map_names, inplace=True)
    display(filtered)
    
    hier = filtered.set_index(['test_opp', 'mapname']) 
    unstacked = hier.unstack(0)['%score']
    ax = unstacked.plot(kind='bar', title=map_type)
    ax.set_xticklabels( ax.get_xticklabels(), rotation=0) # rotates the labels without changing them
    autolabel(ax)
    #ax.get_figure().show()
    ax.get_figure().savefig(f'/tmp/quadrant_{map_type}_p2.pdf')
    display(unstacked)


/home/anderson/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


---- Regular ----


/home/anderson/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,num_reps,train_opp,test_opp,mapname,reward,train_matches,epsilon,gamma,wins,draws,losses,test_matches,score,%score
24,3,selfplay,A3N,basesWorkers16x16A,winlossdraw,100000,0.1,1,3.666667,0.333333,36.000000,40.0,3.833333,9.583333
36,3,selfplay,A3N,basesWorkers24x24A,winlossdraw,100000,0.1,1,9.000000,0.666667,30.333333,40.0,9.333333,23.333333
45,3,selfplay,A3N,basesWorkers8x8,winlossdraw,100000,0.1,1,10.666667,1.333333,28.000000,40.0,11.333333,28.333333
24,3,selfplay,LR,basesWorkers16x16A,winlossdraw,100000,0.1,1,13.333333,0.000000,26.666667,40.0,13.333333,33.333333
36,3,selfplay,LR,basesWorkers24x24A,winlossdraw,100000,0.1,1,13.333333,0.000000,26.666667,40.0,13.333333,33.333333
45,3,selfplay,LR,basesWorkers8x8,winlossdraw,100000,0.1,1,13.333333,0.000000,26.666667,40.0,13.333333,33.333333
24,3,selfplay,PS,basesWorkers16x16A,winlossdraw,100000,0.1,1,12.666667,0.000000,27.333333,40.0,12.666667,31.666667
36,3,selfplay,PS,basesWorkers24x24A,winlossdraw,100000,0.1,1,1.666667,0.000000,38.333333,40.0,1.666667,4.166667
45,3,selfplay,PS,basesWorkers8x8,winlossdraw,100000,0.1,1,27.333333,8.333333,4.333333,40.0,31.500000,78.750000
24,3,selfplay,SSS,basesWorkers16x16A,winlossdraw,100000,0.1,1,1.000000,0.000000,39.000000,40.0,1.000000,2.500000


test_opp,LR,WR,PS,ST,A3N,SSS
mapname,,,,,,
basesWorkers16x16A,33.333333,33.333333,31.666667,45.000000,9.583333,2.500000
basesWorkers24x24A,33.333333,66.666667,4.166667,66.666667,23.333333,0.000000
basesWorkers8x8,33.333333,16.666667,78.750000,32.083333,28.333333,32.083333


---- Split ----


/home/anderson/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,num_reps,train_opp,test_opp,mapname,reward,train_matches,epsilon,gamma,wins,draws,losses,test_matches,score,%score
0,3,selfplay,A3N,FourBasesWorkers8x8,winlossdraw,100000,0.1,1,23.666667,0.000000,16.333333,40.0,23.666667,59.166667
6,3,selfplay,A3N,TwoBasesBarracks16x16,winlossdraw,100000,0.1,1,11.333333,0.000000,28.666667,40.0,11.333333,28.333333
12,3,selfplay,A3N,NoWhereToRun9x8,winlossdraw,100000,0.1,1,0.000000,11.000000,29.000000,40.0,5.500000,13.750000
18,3,selfplay,A3N,DoubleGame24x24,winlossdraw,100000,0.1,1,0.666667,22.666667,16.666667,40.0,12.000000,30.000000
0,3,selfplay,LR,FourBasesWorkers8x8,winlossdraw,100000,0.1,1,26.666667,0.000000,13.333333,40.0,26.666667,66.666667
6,3,selfplay,LR,TwoBasesBarracks16x16,winlossdraw,100000,0.1,1,13.333333,0.000000,26.666667,40.0,13.333333,33.333333
12,3,selfplay,LR,NoWhereToRun9x8,winlossdraw,100000,0.1,1,0.000000,40.000000,0.000000,40.0,20.000000,50.000000
18,3,selfplay,LR,DoubleGame24x24,winlossdraw,100000,0.1,1,13.333333,13.333333,13.333333,40.0,20.000000,50.000000
0,3,selfplay,PS,FourBasesWorkers8x8,winlossdraw,100000,0.1,1,28.000000,0.000000,12.000000,40.0,28.000000,70.000000
6,3,selfplay,PS,TwoBasesBarracks16x16,winlossdraw,100000,0.1,1,13.333333,0.000000,26.666667,40.0,13.333333,33.333333


test_opp,LR,WR,PS,ST,A3N,SSS
mapname,,,,,,
FourBasesWorkers8x8,66.666667,0.000000,70.000000,77.916667,59.166667,0.000000
NoWhereToRun9x8,50.000000,66.666667,0.000000,57.916667,13.750000,64.583333
TwoBasesBarracks16x16,33.333333,66.666667,33.333333,30.000000,28.333333,17.500000
DoubleGame24x24,50.000000,16.666667,25.833333,50.416667,30.000000,19.166667


In [ ]:
'''
This cell is to analyse MAB experiments
'''
mpl.use('TkAgg')
from collections import OrderedDict

filters = {
    'test_pos': 0, 'feature': 'Per-quadrant'
}

opponents = ['LightRush', 'WorkerRush']

# considers only b=0
data_array = [prepare_dataframe(f'../results/mab/{o}_avg.csv', filters=filters) for o in opponents]
df = pd.concat(data_array)

# plot-friendly opponent names
opp_mapping = OrderedDict({
    'LightRush': 'LR', 'WorkerRush': 'WR', # baselines
    'PuppetSearchMCTS': 'PS', 'StrategyTactics': 'ST', # puppet-based 
    'A3N': 'A3N', 'SSSmRTS': 'SSS', # SOTA (A3N is mapped to itself because this is used for sorting too)
})
df.replace(opp_mapping, inplace=True)

# sorts the opponents -- hint by: https://stackoverflow.com/a/27255567/1251716
df.test_opp = df.test_opp.astype("category")
df.test_opp.cat.set_categories(opp_mapping.values(), inplace=True)
#display(df)
#df = df.loc[['basesWorkers8x8', 'basesWorkers16x16A', 'basesWorkers32x32A']] ## attempt to sort map names

hier = df.set_index(['test_opp', 'mapname']) 
unstacked = hier.unstack(0)['%score']
ax = unstacked.plot(kind='bar',title='All maps')
#ax.get_figure().show()
#display(unstacked)

#now, try to separate 'regular' vs 'split' maps
all_maps = {
    'Regular': ['basesWorkers8x8', 'basesWorkers16x16A', 'basesWorkers24x24A', 'basesWorkers32x32A'],
    #'Split': ['FourBasesWorkers8x8', 'NoWhereToRun9x8', 'TwoBasesBarracks16x16', 'DoubleGame24x24'] 
}

for map_type, map_names in all_maps.items():
    print(f'---- {map_type} ----')
    filtered = df[df['mapname'].isin(map_names)]  # isin hint: https://stackoverflow.com/a/12098586/1251716
    # sorts the filtered dataframe by the order given in map_names
    filtered.mapname = filtered.mapname.astype("category")
    filtered.mapname.cat.set_categories(map_names, inplace=True)
    #display(filtered)
    
    hier = filtered.set_index(['test_opp', 'mapname']) 
    unstacked = hier.unstack(0)['%score']
    display(unstacked)
    ''' #commenting plot as the table is enough
    ax = unstacked.plot(kind='bar', title=map_type)
    ax.set_xticklabels( ax.get_xticklabels(), rotation=0) # rotates the labels without changing them
    autolabel(ax)
    ax.get_figure().show()
    #ax.get_figure().savefig(f'/tmp/quadrant_{map_type}_p0.pdf')
    '''
